In [1]:
import pandas as pd
import ast
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

### Load Data

In [21]:
class RedDataset(Dataset):
    def __init__(self, datafile, train):
        self.data = pd.read_csv(datafile)
        self.data = self.data[self.data['train'] == train]

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        item = self.data.loc[idx]
        print(item)
        return torch.tensor(ast.literal_eval(item['data'])), torch.tensor(item['label'])

In [22]:
train_data = RedDataset('RedData.csv', train=True)
test_data = RedDataset('RedData.csv', train=False)

loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

### Model architecture

In [ ]:
class RedModel(nn.Module):
    def __init__(self):
        super(RedModel, self).__init__()
        self.linear1 = nn.Linear()
        self.relu2 = nn.ReLU()
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.relu1(x1)
        x3 = self.pool1(x2)
        x4 = self.conv2(x3)
        x5 = self.relu2(x4)
        x6 = self.pool2(x5)
        x7 = x6.view(x6.size(0), -1)
        output = self.out(x7)
        return {
            'in': x,
            'out': output,
            'conv1': x1,
            'relu1': x2,
            'pool1': x3,
            'conv2': x4,
            'relu2': x5,
            'pool2': x6,
            'x7': x7
        }

### Train model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

cnn = RedModel()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)
num_epochs = 10


def train(num_epochs, cnn, loaders):

    cnn.train()

    # Train the model
    total_step = len(loaders['train'])

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):

            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch youtput = cnn(b_x)[0]

            results = cnn(b_x)['out']
            loss = loss_func(results, b_y)

            # clear gradients for this training step
            optimizer.zero_grad()

            # backpropagation, compute gradients
            loss.backward()                # apply gradients
            optimizer.step()

            if (i + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                pass

        pass

    pass


train(num_epochs, cnn, loaders)


def test():
    cnn.eval()
    with torch.no_grad():
        for images, labels in loaders['test']:
            results = cnn(images)
            pred_y = torch.max(results['out'], 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass

    print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    pass


test()

torch.save(cnn.state_dict(), 'cnn.pt')